# Day 35: Structured Logging for LLM Services - Part 1

Structured logging is essential for observability in production LLM systems. This notebook demonstrates how to implement comprehensive logging for LLM services.

## Overview
1. Setting up structured logging
2. LLM-specific log fields
3. Log aggregation and analysis
4. Performance impact measurement

In [ ]:
# Install required packages
!pip install -q structlog python-json-logger fastapi uvicorn

In [ ]:
import json
import time
import uuid
import logging
import structlog
from datetime import datetime
from typing import Dict, Any, Optional
from contextlib import contextmanager

## 1. Setting Up Structured Logging

We'll configure structured logging with JSON output for easy parsing and analysis.

In [ ]:
def setup_structured_logging():
    """Configure structured logging with JSON output."""
    
    # Configure structlog
    structlog.configure(
        processors=[
            structlog.stdlib.filter_by_level,
            structlog.stdlib.add_logger_name,
            structlog.stdlib.add_log_level,
            structlog.stdlib.PositionalArgumentsFormatter(),
            structlog.processors.TimeStamper(fmt="iso"),
            structlog.processors.StackInfoRenderer(),
            structlog.processors.format_exc_info,
            structlog.processors.UnicodeDecoder(),
            structlog.processors.JSONRenderer()
        ],
        context_class=dict,
        logger_factory=structlog.stdlib.LoggerFactory(),
        wrapper_class=structlog.stdlib.BoundLogger,
        cache_logger_on_first_use=True,
    )
    
    # Configure standard logging
    logging.basicConfig(
        format="%(message)s",
        stream=None,
        level=logging.INFO,
    )
    
    return structlog.get_logger()

# Initialize structured logger
logger = setup_structured_logging()
print("Structured logging configured")

## 2. LLM-Specific Logging Context

Create logging context managers for LLM operations.

In [ ]:
class LLMLogContext:
    """Context manager for LLM request logging."""
    
    def __init__(self, logger, operation: str, **kwargs):
        self.logger = logger
        self.operation = operation
        self.request_id = str(uuid.uuid4())
        self.start_time = None
        self.context = {
            'request_id': self.request_id,
            'operation': operation,
            **kwargs
        }
    
    def __enter__(self):
        self.start_time = time.time()
        self.bound_logger = self.logger.bind(**self.context)
        self.bound_logger.info("Operation started", **self.context)
        return self.bound_logger
    
    def __exit__(self, exc_type, exc_val, exc_tb):
        duration = time.time() - self.start_time
        
        if exc_type is None:
            self.bound_logger.info(
                "Operation completed",
                duration_ms=duration * 1000,
                status="success"
            )
        else:
            self.bound_logger.error(
                "Operation failed",
                duration_ms=duration * 1000,
                status="error",
                error_type=exc_type.__name__,
                error_message=str(exc_val)
            )

@contextmanager
def llm_request_context(logger, tenant_id: str, model_name: str, **kwargs):
    """Context manager for LLM request logging."""
    with LLMLogContext(
        logger, 
        "llm_inference",
        tenant_id=tenant_id,
        model_name=model_name,
        **kwargs
    ) as log:
        yield log

print("LLM logging context managers created")

## 3. Mock LLM Service with Logging

Create a mock LLM service that demonstrates comprehensive logging.

In [ ]:
class LoggedLLMService:
    """Mock LLM service with comprehensive logging."""
    
    def __init__(self, model_name: str = "gpt-3.5-turbo"):
        self.model_name = model_name
        self.logger = structlog.get_logger("llm_service")
        self.request_count = 0
    
    def generate(self, prompt: str, tenant_id: str, max_tokens: int = 100, 
                temperature: float = 0.7) -> Dict[str, Any]:
        """Generate text with comprehensive logging."""
        
        self.request_count += 1
        
        with llm_request_context(
            self.logger,
            tenant_id=tenant_id,
            model_name=self.model_name,
            input_tokens=len(prompt.split()),
            max_tokens=max_tokens,
            temperature=temperature,
            request_number=self.request_count
        ) as log:
            
            # Log input validation
            if len(prompt.strip()) == 0:
                log.warning("Empty prompt received")
                raise ValueError("Prompt cannot be empty")
            
            if max_tokens <= 0:
                log.warning("Invalid max_tokens", max_tokens=max_tokens)
                raise ValueError("max_tokens must be positive")
            
            # Log model loading (mock)
            log.info("Loading model", model_name=self.model_name)
            time.sleep(0.1)  # Simulate model loading
            
            # Log tokenization
            input_tokens = len(prompt.split())
            log.info("Tokenization complete", input_token_count=input_tokens)
            
            # Log inference start
            inference_start = time.time()
            log.info("Starting inference")
            
            # Simulate inference
            time.sleep(max_tokens * 0.01)  # Simulate generation time
            generated_text = f" This is a generated response with {max_tokens} tokens."
            output_tokens = len(generated_text.split())
            
            inference_time = time.time() - inference_start
            
            # Log inference completion
            log.info(
                "Inference complete",
                output_token_count=output_tokens,
                total_tokens=input_tokens + output_tokens,
                inference_time_ms=inference_time * 1000,
                tokens_per_second=output_tokens / inference_time if inference_time > 0 else 0
            )
            
            # Calculate cost (mock)
            cost = (input_tokens + output_tokens) * 0.001
            log.info("Cost calculated", cost_usd=cost)
            
            return {
                'text': generated_text,
                'usage': {
                    'prompt_tokens': input_tokens,
                    'completion_tokens': output_tokens,
                    'total_tokens': input_tokens + output_tokens
                },
                'cost': cost,
                'inference_time': inference_time
            }

# Initialize service
llm_service = LoggedLLMService()
print("Logged LLM service initialized")

## 4. Testing the Logged Service

Let's test our service and see the structured logs in action.

In [ ]:
# Test successful request
print("=== Testing Successful Request ===")
try:
    result = llm_service.generate(
        prompt="What is artificial intelligence?",
        tenant_id="tenant_123",
        max_tokens=20,
        temperature=0.7
    )
    print(f"\nResult: {result['text']}")
    print(f"Usage: {result['usage']}")
except Exception as e:
    print(f"Error: {e}")

print("\n" + "="*50 + "\n")

# Test error case
print("=== Testing Error Case ===")
try:
    result = llm_service.generate(
        prompt="",  # Empty prompt
        tenant_id="tenant_456",
        max_tokens=10
    )
except Exception as e:
    print(f"Expected error caught: {e}")

## 5. Log Analysis and Aggregation

Demonstrate how to analyze the structured logs.

In [ ]:
import io
import sys
from collections import defaultdict

class LogAnalyzer:
    """Analyze structured logs for insights."""
    
    def __init__(self):
        self.logs = []
        self.original_stdout = sys.stdout
        self.log_capture = io.StringIO()
    
    def start_capture(self):
        """Start capturing logs."""
        sys.stdout = self.log_capture
    
    def stop_capture(self):
        """Stop capturing logs and parse them."""
        sys.stdout = self.original_stdout
        log_content = self.log_capture.getvalue()
        
        # Parse JSON logs
        for line in log_content.strip().split('\n'):
            if line.strip():
                try:
                    log_entry = json.loads(line)
                    self.logs.append(log_entry)
                except json.JSONDecodeError:
                    continue
    
    def analyze(self):
        """Analyze captured logs."""
        if not self.logs:
            print("No logs to analyze")
            return
        
        # Basic statistics
        total_logs = len(self.logs)
        log_levels = defaultdict(int)
        operations = defaultdict(int)
        tenants = defaultdict(int)
        
        # Performance metrics
        durations = []
        token_counts = []
        costs = []
        
        for log in self.logs:
            # Count log levels
            level = log.get('level', 'unknown')
            log_levels[level] += 1
            
            # Count operations
            operation = log.get('operation')
            if operation:
                operations[operation] += 1
            
            # Count tenants
            tenant = log.get('tenant_id')
            if tenant:
                tenants[tenant] += 1
            
            # Collect performance metrics
            if 'duration_ms' in log:
                durations.append(log['duration_ms'])
            
            if 'total_tokens' in log:
                token_counts.append(log['total_tokens'])
            
            if 'cost_usd' in log:
                costs.append(log['cost_usd'])
        
        # Print analysis
        print(f"=== Log Analysis ===")
        print(f"Total log entries: {total_logs}")
        
        print(f"\nLog levels:")
        for level, count in log_levels.items():
            print(f"  {level}: {count}")
        
        print(f"\nOperations:")
        for op, count in operations.items():
            print(f"  {op}: {count}")
        
        print(f"\nTenants:")
        for tenant, count in tenants.items():
            print(f"  {tenant}: {count}")
        
        if durations:
            print(f"\nPerformance Metrics:")
            print(f"  Avg duration: {sum(durations)/len(durations):.2f}ms")
            print(f"  Max duration: {max(durations):.2f}ms")
            print(f"  Min duration: {min(durations):.2f}ms")
        
        if token_counts:
            print(f"  Avg tokens: {sum(token_counts)/len(token_counts):.1f}")
        
        if costs:
            print(f"  Total cost: ${sum(costs):.4f}")
            print(f"  Avg cost per request: ${sum(costs)/len(costs):.4f}")

# Test log analysis
analyzer = LogAnalyzer()
analyzer.start_capture()

# Generate some test requests
test_prompts = [
    "What is machine learning?",
    "Explain neural networks",
    "How does AI work?"
]

for i, prompt in enumerate(test_prompts):
    try:
        llm_service.generate(
            prompt=prompt,
            tenant_id=f"tenant_{i % 2 + 1}",
            max_tokens=15 + i * 5
        )
    except Exception:
        pass

analyzer.stop_capture()
analyzer.analyze()

## 6. Performance Impact of Logging

Measure the performance impact of structured logging.

In [ ]:
def benchmark_logging_impact(num_requests=10):
    """Benchmark the performance impact of logging."""
    
    # Simple service without logging
    class SimpleLLMService:
        def generate(self, prompt, max_tokens=100):
            time.sleep(max_tokens * 0.01)  # Simulate generation
            return f"Generated response with {max_tokens} tokens"
    
    simple_service = SimpleLLMService()
    
    # Benchmark without logging
    start_time = time.time()
    for i in range(num_requests):
        simple_service.generate(f"Test prompt {i}", max_tokens=20)
    no_logging_time = time.time() - start_time
    
    # Benchmark with logging (capture output to avoid printing)
    original_stdout = sys.stdout
    sys.stdout = io.StringIO()
    
    start_time = time.time()
    for i in range(num_requests):
        try:
            llm_service.generate(
                prompt=f"Test prompt {i}",
                tenant_id="benchmark_tenant",
                max_tokens=20
            )
        except Exception:
            pass
    with_logging_time = time.time() - start_time
    
    sys.stdout = original_stdout
    
    # Calculate overhead
    overhead = with_logging_time - no_logging_time
    overhead_percent = (overhead / no_logging_time) * 100
    
    print(f"=== Logging Performance Impact ===")
    print(f"Requests: {num_requests}")
    print(f"Without logging: {no_logging_time:.3f}s")
    print(f"With logging: {with_logging_time:.3f}s")
    print(f"Overhead: {overhead:.3f}s ({overhead_percent:.1f}%)")
    print(f"Per-request overhead: {(overhead/num_requests)*1000:.2f}ms")

# Run benchmark
benchmark_logging_impact(20)

## 7. Production Logging Configuration

Create a production-ready logging configuration.

In [ ]:
# Production logging configuration
production_logging_config = """
# production_logging.py
import os
import logging
import structlog
from pythonjsonlogger import jsonlogger

def setup_production_logging():
    """Setup production-ready structured logging."""
    
    # Get log level from environment
    log_level = os.getenv('LOG_LEVEL', 'INFO').upper()
    
    # Configure JSON formatter
    json_formatter = jsonlogger.JsonFormatter(
        fmt='%(asctime)s %(name)s %(levelname)s %(message)s',
        datefmt='%Y-%m-%dT%H:%M:%S'
    )
    
    # Configure handler
    handler = logging.StreamHandler()
    handler.setFormatter(json_formatter)
    
    # Configure root logger
    logging.basicConfig(
        level=getattr(logging, log_level),
        handlers=[handler],
        format='%(message)s'
    )
    
    # Configure structlog
    structlog.configure(
        processors=[
            structlog.stdlib.filter_by_level,
            structlog.stdlib.add_logger_name,
            structlog.stdlib.add_log_level,
            structlog.processors.TimeStamper(fmt="iso"),
            structlog.processors.add_log_level,
            structlog.processors.JSONRenderer()
        ],
        context_class=dict,
        logger_factory=structlog.stdlib.LoggerFactory(),
        wrapper_class=structlog.stdlib.BoundLogger,
        cache_logger_on_first_use=True,
    )
    
    return structlog.get_logger()

# Environment variables for configuration
LOGGING_CONFIG = {
    'LOG_LEVEL': 'INFO',
    'LOG_FORMAT': 'json',
    'LOG_OUTPUT': 'stdout',
    'SERVICE_NAME': 'llm-service',
    'SERVICE_VERSION': '1.0.0',
    'ENVIRONMENT': 'production'
}

# Logging best practices:
# 1. Use structured logging with consistent field names
# 2. Include correlation IDs for request tracing
# 3. Log at appropriate levels (DEBUG, INFO, WARN, ERROR)
# 4. Include performance metrics in logs
# 5. Sanitize sensitive information
# 6. Use log sampling for high-volume operations
# 7. Configure log rotation and retention
"""

# Write production config
with open("production_logging.py", "w") as f:
    f.write(production_logging_config)

print("Production logging configuration created: production_logging.py")
print("\nKey Features:")
print("- JSON structured output")
print("- Environment-based configuration")
print("- Correlation ID support")
print("- Performance metrics logging")
print("- Error context preservation")

## Conclusion

Structured logging provides essential observability for LLM services:

1. **Consistent Format**: JSON logs enable easy parsing and analysis
2. **Rich Context**: Request IDs, tenant info, and performance metrics
3. **Error Tracking**: Comprehensive error logging with context
4. **Performance Monitoring**: Built-in timing and resource usage tracking

**Best Practices**:
- Use correlation IDs for request tracing
- Include relevant business context
- Monitor logging performance impact
- Implement log sampling for high-volume operations

Next, we'll explore metrics collection and monitoring for LLM services.